In [5]:
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [6]:
# train_emoji data and test emoji data is Attached in the repo
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [7]:
# checking the data by printing first 5 entries
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [8]:
# checking the data by printing first 5 entries
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [9]:
# Creating the Dictionary of some emoji's with key a number and value is emoji
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2:":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [10]:
# printing the emoji icon by emojifying each emoji
for ix in emoji_dict.keys():
    print(ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [11]:
# Creating the training and testing data

X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print(("-------------------------"))
print(X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [14]:
# Splitting the train data from sentences to list of words
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()
    
# Splitting the test data from sentences to list of words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

# Converting the labels into categorical Form
Y_train = np_utils.to_categorical(Y_train)

AttributeError: 'list' object has no attribute 'split'

In [15]:
print(X_train[0],Y_train[0])
type(X_train)

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


pandas.core.series.Series

In [14]:
# To check what's the maximum length exist in the training data
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [15]:
# To check what's the maximum length exist in the testing data
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [28]:
# Creating the Embedding dictionary with key = word and value = list of words
embeddings_index = {}

f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
#     print(values)
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [18]:
# Checking the length of each vector
embeddings_index["i"].shape

(50,)

In [19]:
# Using Scipy library to import the embedding matrix
from scipy import spatial

# Checking the cosine similarity of happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [20]:
# Checking the cosine similarity of india and delhi
spatial.distance.cosine(embeddings_index["india"], embeddings_index["delhi"])

0.18572336435317993

In [21]:
# Checking the cosine similarity of france and paris
spatial.distance.cosine(embeddings_index["france"], embeddings_index["paris"])

0.19746702909469604

In [22]:
# Filling the Embedding Matrix

embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]        

In [29]:
print(embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


### Using RNN 

In [30]:
# A simple RNN network to classify the emoji class from a input Sentence

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Setting Loss, Optimizer of the Model 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
# Training of the Model

hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
132/132 [==============================] - 1s 5ms/step - loss: 1.8833 - acc: 0.2500
Epoch 2/50
132/132 [==============================] - 0s 476us/step - loss: 1.7785 - acc: 0.2197
Epoch 3/50
132/132 [==============================] - 0s 489us/step - loss: 1.6567 - acc: 0.2955
Epoch 4/50
132/132 [==============================] - 0s 509us/step - loss: 1.6565 - acc: 0.2273
Epoch 5/50
132/132 [==============================] - 0s 498us/step - loss: 1.5134 - acc: 0.3561
Epoch 6/50
132/132 [==============================] - 0s 546us/step - loss: 1.5075 - acc: 0.3636
Epoch 7/50
132/132 [==============================] - 0s 639us/step - loss: 1.5513 - acc: 0.3712
Epoch 8/50
132/132 [==============================] - 0s 519us/step - loss: 1.4025 - acc: 0.4091
Epoch 9/50
132/132 [==============================] - 0s 543us/step - loss: 1.3645 - acc: 0.4773
Epoch 10/50
132/132 [==============================] - 0s 621us/step - loss: 1.3342 - acc: 0.4318
Epoch 11/50
132/132 [==========

In [33]:
# prediction from the trained model
pred = model.predict_classes(embedding_matrix_test)

In [34]:
# Calculating the score of the algorithm

float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.5

In [36]:
# printing the sentences with the predicted emoji and the labelled emoji

for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print(test[0][ix],end=" ")
        print(emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print(emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

3
['she', 'got', 'me', 'a', 'present'] 😄 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴 ❤
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤ 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
15
['This', 'stupid', 'grader', 'is', 'not', 'working'] 😄 😞
18
['stop', 'messing', 'around'] ❤ 😞
20
['I', 'love', 'taking', 'breaks'] 🍴 ❤
21
['you', 'brighten', 'my', 'day'] 🍴 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 😞 ❤
28
['enjoy', 'your', 'break'] ❤ 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
32
['My', 'life', 'is', 'so', 'boring'] 😄 😞
35
['he', 'can', 'pitch', 'really', 'well'] 😞 ⚾
36
['dance', 'with', 'me'] 😞 😄
38
['See', 'you', 'at', 'the', 'restaurant'] 😄 🍴
40
['I', 'will', 'go', 'dance'] ⚾ 😄
41
['I', 'like', 'your', 'jacket'] ❤ 😄
45
['I', 'love', 'to', 'the', 'stars', 'and', 'back'] 😞 ❤
46
['What', 'you', 'did', 'was', 'awesome'] ⚾ 😄
48


In [55]:
# Predicting for Our random sentence

x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]

In [57]:
model.predict_classes(x_)

array([3])

### Using LSTM 

In [37]:

model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
132/132 [==============================] - 2s 13ms/step - loss: 1.5877 - acc: 0.2652
Epoch 2/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5185 - acc: 0.3333
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4466 - acc: 0.4167
Epoch 4/50
132/132 [==============================] - 0s 2ms/step - loss: 1.4128 - acc: 0.4394
Epoch 5/50
132/132 [==============================] - 0s 2ms/step - loss: 1.3376 - acc: 0.4242
Epoch 6/50
132/132 [==============================] - 0s 2ms/step - loss: 1.2094 - acc: 0.5606
Epoch 7/50
132/132 [==============================] - 0s 2ms/step - loss: 1.0628 - acc: 0.6364
Epoch 8/50
132/132 [==============================] - 0s 2ms/step - loss: 1.0022 - acc: 0.6136
Epoch 9/50
132/132 [==============================] - 0s 2ms/step - loss: 0.8686 - acc: 0.6591
Epoch 10/50
132/132 [==============================] - 0s 2ms/step - loss: 0.8543 - acc: 0.6894
Epoch 11/50
132/132 [===========================

In [40]:
pred = model.predict_classes(embedding_matrix_test)

In [41]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.625

In [43]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print(test[0][ix],end=" ")
        print(emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print(emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

2
['he', 'got', 'a', 'raise'] 😞 😄
3
['she', 'got', 'me', 'a', 'present'] 😄 ❤
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄 ❤
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤ 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
18
['stop', 'messing', 'around'] ⚾ 😞
20
['I', 'love', 'taking', 'breaks'] 😞 ❤
21
['you', 'brighten', 'my', 'day'] ❤ 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
28
['enjoy', 'your', 'break'] 😞 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
32
['My', 'life', 'is', 'so', 'boring'] ❤ 😞
40
['I', 'will', 'go', 'dance'] ⚾ 😄
41
['I', 'like', 'your', 'jacket'] ❤ 😄
49
['go', 'away'] ⚾ 😞
54
['You', 'totally', 'deserve', 'this', 'prize'] 😞 😄
55
['I', 'did', 'not', 'have', 'breakfast'] 🍴 😞
